This script analyze the output of O3 box model with source apportionment technique. The goal of this script is to find the O3 contribution from different source of VOCs from the box model for Redlands, CA

In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import join, isfile

In [ ]:
df_merge = pd.DataFrame(columns=['Date','Time','O3_X1','O3_X2','O3_X3',
                                               'O3_X4','O3_X5','O3_X6',
                                               'O3_X7','O3_X8','O3_X9'])
folderpath = '../../data/intermediate/box_model_output_bgX/'
filelist = [f for f in listdir(folderpath) if 
            isfile(join(folderpath, f))]
filelist = [s for s in filelist if 'results' in s ]

for i in filelist:
    filepath = folderpath+i
    date = i[8:18]
    print(filepath)
    
    with open(filepath) as f:    
        contents = f.readlines()
    
    # get O3 concentration in list
    list_O3 = [s for s in contents if " O3 " in s]
    list_O3X1 = [s for s in contents if " O3_X1" in s]
    list_O3X2 = [s for s in contents if " O3_X2" in s]
    list_O3X3 = [s for s in contents if " O3_X3" in s]
    list_O3X4 = [s for s in contents if " O3_X4" in s]
    list_O3X5 = [s for s in contents if " O3_X5" in s]
    list_O3X6 = [s for s in contents if " O3_X6" in s]
    list_O3X7 = [s for s in contents if " O3_X7" in s]
    list_O3X8 = [s for s in contents if " O3_X8" in s]
    list_O3X9 = [s for s in contents if " O3_X9" in s]

    # create df to store O3 source apportionment results in one model run
    df = pd.DataFrame(columns=['Date','Time','O3_X1','O3_X2','O3_X3',
                                             'O3_X4','O3_X5','O3_X6',
                                             'O3_X7','O3_X8','O3_X9'])
    df['Time'] = np.arange(0,490,10)
    df['Date'] = date

    # save O3 source concentration (O3X1, O3X2, O3X3) to df
    for j in range(len(list_O3)):
        df.loc[j,'O3_X1'] = list_O3X1[j][33:33+13]
        df.loc[j,'O3_X2'] = list_O3X2[j][33:33+13]
        df.loc[j,'O3_X3'] = list_O3X3[j][33:33+13]    
        df.loc[j,'O3_X4'] = list_O3X4[j][33:33+13]
        df.loc[j,'O3_X5'] = list_O3X5[j][33:33+13]
        df.loc[j,'O3_X6'] = list_O3X6[j][33:33+13]    
        df.loc[j,'O3_X7'] = list_O3X7[j][33:33+13]
        df.loc[j,'O3_X8'] = list_O3X8[j][33:33+13]
        df.loc[j,'O3_X9'] = list_O3X9[j][33:33+13]    
    df_merge = pd.concat([df_merge, df])
    
# change vector type 
df_merge.loc[:,'Time':] = df_merge.loc[:,'Time':].astype(float)